In [1]:
import numpy as np
import tensorflow as tf
import uproot
import awkward as ak
import vector
import numba as nb
import os
import fnmatch
from dnn_tau import Dnn_tau
from kinematic import *
from utils import isolate_int, normalize
import pandas as pd
from numbers import Number
from utils import normalize, split_dataset, bucketize
from data_extractor import Data_extractor_v1

In [2]:
path = "/data/hnl/prompt_tau/anatuple/nanoV10/TEST6/ttm_DeepTau2p5/"
extractor = Data_extractor_v1('ttm')
data = extractor(path)

In [3]:
N = len(data['event'])
n_bkg = N-sum([data['signal_label'][i] for i in range(len(data['signal_label']))])

data_norm = normalize(pd.DataFrame(data), 'mass_hyp', n_bkg)
data_norm = normalize(data_norm, 'signal_label', n_bkg)
print("Total background weight : ", sum(data_norm['genWeight'][i] for i in range(N) if data['signal_label'][i] == 0))
print("Total signal weight : ", sum(data_norm['genWeight'][i] for i in range(N) if data['signal_label'][i] == 1))
print("Number of background events : ", n_bkg)
data_norm = normalize(data_norm, 'channel', n_bkg)

Total background weight :  534759.9999999966
Total signal weight :  534759.9999999988
Number of background events :  534760.0


In [5]:
data_processed, channel_indices = bucketize(data_norm, 'channel')